In [1]:
# If you don't already have HomotopyContinuation.jl installed this might take a while
# This code depends on the https://github.com/ACSVMath/ACSVHomotopy package
using Pkg
Pkg.activate(".")
using ACSVHomotopy
@polyvar x y z

  Activating project at `~/ACSVHomotopy`


(x, y, z)

## Non-Crossing Configurations

In [2]:
# non-crossing graphs
H = (y^2*x + 6*y*x + 8*x - 1)
@time find_min_crits(H)

 46.720010 seconds (87.24 M allocations: 4.335 GiB, 2.31% gc time, 84.69% compilation time)


1-element Vector{Vector{Float64}}:
 [0.03033008588991066, 2.828427124746189]

In [3]:
# the heuristic method provides several orders of magnitude performance improvement
@time MCP = find_min_crits(H; approx_crit=true)

 12.084447 seconds (28.58 M allocations: 1.345 GiB, 2.01% gc time, 98.11% compilation time)


1-element Vector{Vector{Float64}}:
 [0.03033008588991066, 2.828427124746189]

In [4]:
println("exponential growth: ", 1/prod(MCP[1]))
println("expected: ", 6 + 4*sqrt(2))

exponential growth: 11.656854249492378
expected: 11.65685424949238


In [5]:
# non-crossing connected graphs
H = (y^5*x^3 + 3*y^4*x^3 + 3*y^3*x^3 + 3*y^3*x^2 + y^2*x^3 + 6*y^2*x^2 + y^2*x + 3*y*x^2 + 5*y*x + 4*x - 1)
@time MCP = find_min_crits(H; approx_crit=true)

 53.599970 seconds (28.75 M allocations: 1.558 GiB, 1.02% gc time, 25.57% compilation time)


1-element Vector{Vector{Float64}}:
 [0.05391937820329315, 1.7846096908265279]

In [6]:
println("exponential growth: ", 1/prod(MCP[1]))
println("expected: ", 6*sqrt(3))

exponential growth: 10.392304845413264
expected: 10.392304845413264


In [7]:
# non-crossing forests
H = (y^7*x^5 + 21*y^6*x^5 + 147*y^5*x^5 + 7*y^5*x^4 + 343*y^4*x^5 + 98*y^4*x^4 + 2*y^4*x^3 + 343*y^3*x^4 + 44*y^3*x^3 + 210*y^2*x^3 + 10*y^2*x^2 + 82*y*x^2 + 3*y*x + 33*x - 1)
@time MCP = find_min_crits(H; approx_crit=true)

612.443327 seconds (65.71 M allocations: 5.346 GiB, 0.21% gc time, 4.48% compilation time)


1-element Vector{Vector{Float64}}:
 [0.0043199830940064176, 28.144810807432684]

In [8]:
println("exponential growth: ", 1/prod(MCP[1]))
println("expected: ", 8.22469)

exponential growth: 8.224691540977856
expected: 8.22469


In [9]:
# non-crossing trees
H = (y^7*x^5 + 9*y^6*x^5 + 27*y^5*x^5 + 3*y^5*x^4 + 27*y^4*x^5 + 18*y^4*x^4 + 3*y^4*x^3 + 27*y^3*x^4 + 21*y^3*x^3 + 36*y^2*x^3 + 6*y^2*x^2 + 19*y*x^2 + 3*y*x + 12*x - 1)
@time MCP = find_min_crits(H; approx_crit=true)

185.547988 seconds (4.48 M allocations: 232.102 MiB, 0.04% compilation time)


1-element Vector{Vector{Float64}}:
 [0.011368682831512572, 13.031249999999993]

In [10]:
println("exponential growth: ", 1/prod(MCP[1]))
println("expected: ", 27/4)

exponential growth: 6.750000000000002
expected: 6.75


#### this cell may take a long time to execute

In [11]:
# # non-crossing connected graphs (without approximated critical points)
# H = (y^5*x^3 + 3*y^4*x^3 + 3*y^3*x^3 + 3*y^3*x^2 + y^2*x^3 + 6*y^2*x^2 + y^2*x + 3*y*x^2 + 5*y*x + 4*x - 1)
# @time MCP = find_min_crits(H)

## Lattice Walks
In these examples we also compute subexponential growth using the smooth ACSV asymptotics theorem.

In [ ]:
using LinearAlgebra
using Symbolics

# numbers smaller than this are *probably* zero
# we do numerical computations here, but these operation can be done symbolically
ϵ = 1e-14
function asymptotics(G, H, MCP)
    z = variables(H)
    d = length(z)
    bases = []
    coeffs = []
    for ζ in MCP
        λ = convert(ComplexF64, ζ[1] * subs(differentiate(H, z[1]), z => ζ))
        # compute the hessian of ϕ (lemma 5.5 of An Invitation to Analytic Combinatorics)
        U = [[convert(ComplexF64, ζ[k]*ζ[l]*subs(differentiate(differentiate(H, z[k]), z[l]), z => ζ))/λ for k in 1:d] for l in 1:d]
        Hes = [[i == j ? convert(ComplexF64, 2 + U[i][j] - U[i][d] - U[j][d] + U[d][d]) : 
                         convert(ComplexF64, 1 + U[i][j] - U[i][d] - U[j][d] + U[d][d]) for i in 1:d-1] for j in 1:d-1]
        Hes = reduce(hcat, Hes)
        C₀ = -G(z => ζ) / (ζ[d]*differentiate(H, z[d])(z => ζ))
        
        # we know these coefficients are real in these cases
        coeff = convert(Real, (2π)^((1-d)/2) / sqrt(det(Hes)) * C₀)
        push!(bases, convert(Real, 1/prod(ζ)))
        push!(coeffs, coeff < ϵ ? 0 : coeff)
    end
    asm(n) = sum(coeffs[k]*n^((1-d)/2)*bases[k]^n for k in 1:length(MCP))
    return asm
end
@variables n

In [13]:
H = 1 - (z*x^2*y^2 + z*x^2 + z*y^2 + z)
@time MCP = find_min_crits(H; approx_crit=true)
println("minimal points: ", MCP)

asm = asymptotics((1+x)*(1+y), H, MCP)
println("result: ", asm(n))
println("expect: ", 2/π*(4^n/n))

 59.638567 seconds (58.25 M allocations: 2.982 GiB, 1.32% gc time, 32.08% compilation time: 44% of which was recompilation)
minimal points: [[-1.0, -1.0, 0.24999999999999994], [-1.0, 1.0, 0.24999999999999994], [1.0, -1.0, 0.24999999999999994], [1.0, 1.0, 0.24999999999999994]]
result: (0.6366197723675815(4.000000000000001^n)) / n
expect: (0.6366197723675814(4^n)) / n


In [14]:
H = 1-(z*x^2*y^2 + z*x*y^2 + z*x^2 + z*y^2 + z*x + z)
@time MCP = find_min_crits(H; approx_crit=true)
println("minimal points: ", MCP)

asm = asymptotics((1+x)*(1+y), H, MCP)
println("result: ", asm(n))
println("expect: ", sqrt(6)/π*(6^n/n))

 57.388850 seconds (25.83 M allocations: 1.287 GiB, 1.12% gc time, 19.16% compilation time: 0% of which was recompilation)
minimal points: [[1.0000000000000002, -1.0000000000000002, 0.16666666666666655], [1.0000000000000002, 1.0000000000000002, 0.16666666666666655]]
result: (0.7796968012336765(6.000000000000002^n)) / n
expect: (0.779696801233676(6^n)) / n


In [15]:
H = 1-(z*x^2*y^2 + z*x^2*y + z*x*y^2 + z*x^2 + z*y^2 + z*x + z*y + z)
@time MCP = find_min_crits(H; approx_crit=true)
println("minimal points: ", MCP)

asm = asymptotics((1+x)*(1+y), H, MCP)
println("result: ", asm(n))
println("expect: ", 8/3π*(8^n/n))

130.062642 seconds (32.16 M allocations: 1.648 GiB, 0.51% gc time, 10.09% compilation time)
minimal points: [[0.9999999999999998, 0.9999999999999998, 0.12500000000000008]]
result: (0.8488263631567748(7.999999999999998^n)) / n
expect: (0.8488263631567752(8^n)) / n
